In [ ]:
import csv
import gzip
import io
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import math

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_gcp import *

In [ ]:
# Install a particular version of `google-cloud-storage` because (oddly enough)
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

In [ ]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [ ]:
bucket_name = "bucket_ir_102"
base_dir = "postings_gcp"
N = 6348910 # Number of docs in the corpus

# Reading the inverted index.
invertedIndex = InvertedIndex.read_index(base_dir,"index",bucket_name)

In [ ]:
def tf_idf(invertedIndex, doc_length_dict, query, bucket_name, base_dir="postings_gcp"):
    """
    args:
    invertedIndex - inverted index, which allows to get all of the data needed.
    doc_length_dict - a dict that when given the doc_id, gives the length of it.
    query - stemmed and trimmed list of the words that were searched.
    bucket_name - the name of the bucket connected to the inverted index.

    returns a counter:
      key = doc_id
      value = W calculated by the tf-idf
      with the words asked.
    """
    N = 6348910 # Number of docs in the corpus

    # For the return values
    tf_idf_dict = Counter()

    for word in query:
        # Only handle words that are in the dictionary.
        if word in invertedIndex.df:
            term_df = invertedIndex.df[word]
            pl = invertedIndex.read_a_posting_list(base_dir, word, bucket_name)
            # Map it to get -> [(doc_id,W)....]
            pl_map = pl.map(lambda x: (x[0],
                            (x[1]/doc_length_dict[x[0]])*(math.log2(N/term_df))))
            # Update tf_idf_dict with the new values, summing if the key already exists
            tf_idf_dict.update(dict(pl_map))

    return tf_idf_dict